In [1]:
import numpy as np
import pandas as pd

In [2]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [4]:
data = pd.read_csv('Reviews.csv',nrows = 100000)

In [5]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [6]:
data.Text[0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [7]:
data.drop_duplicates(subset=['Text'],inplace=True) 
data.dropna(axis=0,inplace=True) 

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}


In [9]:
stop_words =set( stopwords.words('english'))
def textCleaner(text):
    newString = text.lower()
    newString = re.sub('/\*[()]',' ',newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping.keys() else t for t in newString.split()])
    newString = re.sub('\'s' , '',text)
    newString = re.sub('[^a-zA-Z]'," ",newString)
    newString = re.sub('"','', newString)
    newString = [t for t in newString.split() if not t in stop_words ]
    long_words=[]
    for i in newString:
        if len(i)>=3:                  
            long_words.append(i)   
    return (" ".join(long_words)).strip()

cleaned_text = []
for t in data['Text']:
    cleaned_text.append(textCleaner(t))

In [10]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString


cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data['cleaned_summary'].replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [11]:
data['cleaned_text'][0:10]

0                                   bought several Vitality canned dog food products found good quality The product looks like stew processed meat smells better Labrador finicky appreciates product better
1                                                                  Product arrived labeled Jumbo Salted Peanuts peanuts actually small sized unsalted Not sure error vendor intended represent product Jumbo
2    This confection around centuries light pillowy citrus gelatin nuts case Filberts And cut tiny squares liberally coated powdered sugar And tiny mouthful heaven Not chewy flavorful highly recommend ...
3                                                                       looking secret ingredient Robitussin believe found got addition Root Beer Extract ordered good made cherry soda The flavor medicinal
4                                                                                                                  Great taffy great price There wide assortment yummy taffy Deliver

In [12]:

for i in range(5):
    print("Review:",data['cleaned_text'][i])
    print("Summary:",data['cleaned_summary'][i])
    print("\n")

Review: bought several Vitality canned dog food products found good quality The product looks like stew processed meat smells better Labrador finicky appreciates product better
Summary: good quality dog food 


Review: Product arrived labeled Jumbo Salted Peanuts peanuts actually small sized unsalted Not sure error vendor intended represent product Jumbo
Summary: not as advertised 


Review: This confection around centuries light pillowy citrus gelatin nuts case Filberts And cut tiny squares liberally coated powdered sugar And tiny mouthful heaven Not chewy flavorful highly recommend yummy treat familiar story Lewis The Lion The Witch The Wardrobe treat seduces Edmund selling Brother Sisters Witch
Summary: delight says it all 


Review: looking secret ingredient Robitussin believe found got addition Root Beer Extract ordered good made cherry soda The flavor medicinal
Summary: cough medicine 


Review: Great taffy great price There wide assortment yummy taffy Delivery quick taffy lover 

In [13]:
data['cleaned_summary'] = data['cleaned_summary'].apply(lambda x : '_START_ '+ x + ' _END_')

In [14]:
data['cleaned_summary']

0                                                        _START_ good quality dog food  _END_
1                                                            _START_ not as advertised  _END_
2                                                          _START_ delight says it all  _END_
3                                                               _START_ cough medicine  _END_
4                                                                  _START_ great taffy  _END_
                                                 ...                                         
99995                                                                    _START_ yummy  _END_
99996                                                         _START_ tastes like more  _END_
99997                                                              _START_ great ramen  _END_
99998                                                                    _START_ spicy  _END_
99999    _START_ this spicy noodle cures my cold upset stoma

In [15]:
summary_word_count = []
text_word_count = []
for i in data['cleaned_text']:
    text_word_count.append(len(i.split()))
for i in data['cleaned_summary']:
    summary_word_count.append(len(i.split()))

In [16]:
print(np.mean(text_word_count))

41.660947120608476


In [17]:
max_len_text=80 
max_len_summary=10

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val = train_test_split(data['cleaned_text'],data['cleaned_summary'],test_size = 0.1,random_state = 0,shuffle = True)

In [20]:
x_tr[0]

'bought several Vitality canned dog food products found good quality The product looks like stew processed meat smells better Labrador finicky appreciates product better'

In [21]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

x_tr    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1


In [22]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

In [23]:
y_tr = y_tokenizer.texts_to_sequences(y_tr)
y_val = y_tokenizer.texts_to_sequences(y_val)

In [24]:
y_tr[0]

[1, 13, 14, 2]

In [25]:
y_tr = pad_sequences(y_tr,maxlen = max_len_summary , padding= 'post')
y_val = pad_sequences(y_val,maxlen = max_len_summary ,padding = 'post')
y_vocab_size = len(y_tokenizer.word_index) + 1

In [26]:
y_tr[0]

array([ 1, 13, 14,  2,  0,  0,  0,  0,  0,  0])

In [27]:
y_tokenizer.word_index

{'start': 1,
 'end': 2,
 'great': 3,
 'the': 4,
 'good': 5,
 'for': 6,
 'not': 7,
 'and': 8,
 'it': 9,
 'best': 10,
 'my': 11,
 'but': 12,
 'love': 13,
 'this': 14,
 'to': 15,
 'is': 16,
 'coffee': 17,
 'tea': 18,
 'of': 19,
 'product': 20,
 'delicious': 21,
 'taste': 22,
 'very': 23,
 'dog': 24,
 'like': 25,
 'in': 26,
 'flavor': 27,
 'excellent': 28,
 'food': 29,
 'these': 30,
 'tasty': 31,
 'you': 32,
 'price': 33,
 'yummy': 34,
 'too': 35,
 'with': 36,
 'as': 37,
 'favorite': 38,
 'ever': 39,
 'so': 40,
 'are': 41,
 'on': 42,
 'dogs': 43,
 'loves': 44,
 'stuff': 45,
 'snack': 46,
 'just': 47,
 'better': 48,
 'have': 49,
 'no': 50,
 'chocolate': 51,
 'healthy': 52,
 'awesome': 53,
 'tastes': 54,
 'than': 55,
 'free': 56,
 'nice': 57,
 'them': 58,
 'what': 59,
 'sweet': 60,
 'perfect': 61,
 'at': 62,
 'bad': 63,
 'really': 64,
 'one': 65,
 'quality': 66,
 'yum': 67,
 'all': 68,
 'treat': 69,
 'tasting': 70,
 'wonderful': 71,
 'your': 72,
 'me': 73,
 'buy': 74,
 'cat': 75,
 'be': 76,


In [28]:
from attention import AttentionLayer

In [29]:
latent_dim = 500
encoder_input = Input(shape = (max_len_text,))
embedding1 =Embedding(x_voc_size ,latent_dim,trainable= True)(encoder_input)

lstm1 = LSTM(latent_dim , return_sequences=True , return_state= True)
lstm1_outputs ,lstm1_state_h,lstm_state_c = lstm1(embedding1)

lstm2 = LSTM(latent_dim , return_sequences = True , return_state = True)
lstm2_outputs , lstm2_state_h, lstm2_state_c = lstm2(lstm1_outputs)

lstm3 = LSTM(latent_dim , return_sequences = True , return_state = True)
encoder_outputs , state_h, state_c = lstm3(lstm2_outputs)

decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_vocab_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

decoder_lstm = LSTM(latent_dim , return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

decoder_dense = TimeDistributed(Dense(y_vocab_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

model = Model([encoder_input, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 500)      24915000    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 80, 500), (N 2002000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [30]:
model.compile(optimizer='rmsprop',loss = 'sparse_categorical_crossentropy')

In [31]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [32]:
y_tr[:,:-1][0]

array([ 1, 13, 14,  2,  0,  0,  0,  0,  0])

In [33]:
y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:][0]

array([[13],
       [14],
       [ 2],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0]])

In [37]:
history = model.fit([x_tr,y_tr[:,:-1]] , y_tr.reshape(y_tr.shape[0],y_tr.shape[1],1)[:,1:] , epochs = 10,batch_size=128, validation_data=([x_val,y_val[:,:-1]],y_val.reshape(y_val.shape[0] , y_val.shape[1],1)[:,1:]))

Train on 79516 samples, validate on 8836 samples
Epoch 1/10
79516/79516 [==============================] - 1796s 23ms/sample - loss: 2.8950 - val_loss: 2.5148
Epoch 2/10
79516/79516 [==============================] - 1545s 19ms/sample - loss: 2.5024 - val_loss: 2.3489
Epoch 3/10
79516/79516 [==============================] - 1514s 19ms/sample - loss: 2.3431 - val_loss: 2.2615
Epoch 4/10
79516/79516 [==============================] - 1514s 19ms/sample - loss: 2.2279 - val_loss: 2.2183
Epoch 5/10
79516/79516 [==============================] - 1505s 19ms/sample - loss: 2.1301 - val_loss: 2.1887
Epoch 6/10
79516/79516 [==============================] - 1504s 19ms/sample - loss: 2.0421 - val_loss: 2.1779
Epoch 7/10
79516/79516 [==============================] - 1504s 19ms/sample - loss: 1.9539 - val_loss: 2.1857
Epoch 8/10
79516/79516 [==============================] - 3518s 44ms/sample - loss: 1.8660 - val_loss: 2.2024
Epoch 9/10
79516/79516 [==============================] - 1861s 23ms/sa

In [43]:
history

In [50]:
reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y_tokenizer.word_index

In [48]:
#inf = inference phase
encoder_model = Model(encoder_input,[encoder_outputs,state_h,state_h])

decoder_state_h = Input(shape = (latent_dim,))
decoder_state_c = Input(shape = (latent_dim,))
decoder_hidden_state_input = Input(shape = (max_len_text,latent_dim))

dec_emb2= dec_emb_layer(decoder_inputs)
decoder_outputs2,state_h2,state_c2 = decoder_lstm(dec_emb2 , initial_state=[decoder_state_h,decoder_state_c])

attn_out_inf , attn_state_inf = attn_layer([decoder_hidden_state_input,decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

decoder_outputs2 = decoder_dense(decoder_inf_concat)

decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,decoder_state_h, decoder_state_c],
[decoder_outputs2] + [state_h2, state_c2])

In [89]:
def decode_seq(input_seq):
    enc_out,e_h,e_c = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0,0] = target_word_index['start']
    decoded_sent =''
    stop = False
    while not stop:
        dec_output ,h,c = decoder_model.predict([target_seq] + [enc_out,e_h,e_c])
        index = np.argmax(dec_output)
        cur_word = reverse_target_word_index[index]
        if cur_word != 'end':
            decoded_sent = decoded_sent+ cur_word+" "
        if cur_word == 'end' or len(decoded_sent.split())>max_len_summary:
            stop = True
        target_seq = np.zeros((1,1))
        target_seq[0,0] = index
        e_h,e_c = h,c
    return decoded_sent
        
        

In [124]:
decode_seq(x_tr[0].reshape(1,max_len_text))

'great taste '

In [127]:
x_tokenizer.sequences_to_texts([y_tr[0]])

['like love food the']

In [135]:
def userReview(review):
    #review = input()
    review = 'start ' + textCleaner(review) + ' end'
    review_mat = x_tokenizer.texts_to_sequences([review])
    review_mat =pad_sequences(review_mat,  maxlen=max_len_text, padding='post') 
    print(review_mat)
    return decode_seq([review_mat])
    

In [136]:
userReview('The Food we had enjoyed at the time of dinner. It was really delicious taste with great quality, everything had unique taste which we had ordered, nice arrangement and services from the staff while eating, we found nothing bad about this hotel.')

[[ 484    2   14  405   21  490   17   68    4    7   96  367 1024    4
   111   66 7314 6771 5269  118   46  253  112 4054  397    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]]


'delicious '

In [138]:
userReview('The ambiance was good and the food recommendations were great. We had a traditional Arabian rice and chicken preparation, Kuzhi Manthi. It was our first experience having this dish, and although a bit dry, it tasted great.')

[[  484     2 22403     3    14  4797     7   928 14180   167   125  2152
     37   465   621   317    55   201   174     7   397     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


'good stuff '

In [142]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    return newString

In [144]:
for i in range(10):
  print("Review:",seq2text(x_val[i]))
  print("Original summary:",seq2summary(y_val[i]))
  print("Predicted summary:",decode_seq(x_val[i].reshape(1,max_len_text)))
  print("\n")

Review: ordered salmon thursday january received january the salmon delicious the wooden box nice design used store items future 
Original summary: alaska smokehouse smoked salmon 
Predicted summary: great gift 


Review: drank cold could pleased the coffee high quality arabica always notice coffee arabica robusta sweet without sweet mean compare major bottled coffee brand reason alone you keep fridge work offer people without looking like pushing calories along caffeine for long time stock highly sweetened coffee beverages fridge work then noticed people stopped consuming happened around time lost interest level sweetness recommend drink prefer lower level sugar old school arabica coffee types 
Original summary: it was perfect little sweet without being too sweet 
Predicted summary: great coffee 


Review: variety granola and one ranks top favorites crunch granola coconut main taste sweet overly eat snack craving something fills fast satisfies sugar cravings 
Original summary: great t